# Sensorfusion - Autonomes Fahren


In [409]:
# Imports
import pandas as pd
import numpy as np
from pyds import MassFunction


In [410]:
data=pd.read_csv("DPE16_1.csv", sep=';', decimal=",")

#TODO rename column Bbox_Hoehe

In [411]:
data

,t,Eigengeschwindigkeit(m/s),Bbox_Breite,Bbox_Hoehe,Bbox_links_unten_y,Abstand(m)
0,0.0,22.0,90,99,153,76.5
1,0.1,22.2,90,100,154,77.0
2,0.2,22.2,86,95,154,77.0
3,0.3,22.1,90,100,153,76.5
4,0.4,22.1,90,100,153,76.5
...,...,...,...,...,...,...
99,9.9,22.1,154,171,117,58.5
100,10.0,22.1,157,174,114,57.0
101,10.1,22.0,158,175,116,58.0
102,10.2,22.2,157,174,114,57.0


# Notizen
Jede Zeile entspricht Messung.
Verhältnis Breite/Höhe durch BBox-Werte.
Geschwindigkeit nur über Zeit (min Anzahl an Messwerten?) durch Abstand Differenz - Eigengeschwindigkeit
Beschleunigung über Zeit (sobald 2 Werte für Geschwindigkeit vorhanden: GeschwindigkeitNeu - GeschwindigkeitAlt

Konfidenz höher, je kleiner der Abstand
Omega: 1 - Konfidenz
omega: P L M F



In [412]:
omega = 'plmf'
#Save speed of previous measurement
oldSpeedSaved = None
newSpeedSaved = None


In [413]:
# Higher value means bycicle or motorbike
# lower value means car
# not in the task but we think equal ratios should increase the probability for truck
# values around 1 mean truck
def calcBoundingRatio(height, width):
    ratio = height / width
    
    return ratio
    
    
# Manage saved values of speed to calculate acceleration
# Only called by calcSpeed
def saveSpeed(speed):
    global newSpeedSaved
    global oldSpeedSaved
    if (newSpeedSaved):
        oldSpeedSaved = newSpeedSaved
    newSpeedSaved = speed
    
#calculate speed
def calcSpeed(oldOwnSpeed, newOwnSpeed, oldDistance, newDistance):
    global newSpeedSaved
    # Check if there are null values for distance
    if (pd.isna(newDistance)):
        print('New Distance is NaN')
        print('Do not evaluate speed due to measurement error')
        saveSpeed(None)
        
        return
    
    difDistance = newDistance - oldDistance
    #Use average speed because old and new speed can be slightly different
    ownSpeed = (oldOwnSpeed + newOwnSpeed) / 2
    speed = difDistance / 0.1 + ownSpeed
    saveSpeed(speed)
    
    return speed
    
# calculate acceleration with formula: dif speed / dif time
def calcAcceleration():
    global newSpeedSaved
    global oldSpeedSaved
    #Catch oldSpeed null for first measurement
    if ((oldSpeedSaved == None) | (newSpeedSaved == None)):
        print('Keine Daten zur Beschleunigung vorhanden')
        # Return 0 by guessing, two measurements are required.
        # No problem because acceleration will only have effect if it is high
        return 0
    speedDif = newSpeedSaved - oldSpeedSaved
    acc = speedDif / 0.1
    
    return acc
    
#def calcMasses(oldMeasurement, newMeasurement):
    # Call other calcs
    
testspd1 = calcSpeed(22.2, 22.1, 77.0, 76.5)
print(testspd1)
testspd2 = calcSpeed(22.1, 22.1, 76.5, 76.5)
print(testspd2)
testspd3 = calcSpeed(22.1, 22.1, 76.5, np.nan)
#testspd4 = calcSpeed(22.1, 22.1, 76.5, 76.5)



testacc = calcAcceleration()
print(testacc)

17.15
22.1
New Distance is NaN
Do not evaluate speed due to measurement error
Keine Daten zur Beschleunigung vorhanden
0


In [414]:
def iterateMeasurement(data):
    #for row in data.items():
    for index, row in data.iterrows():
        print('---------------------------------------------------')
        print('Zeitpunkt:', row['t'])
        # Evaluate Ratio
        ratio = calcBoundingRatio(row['Bbox_Hoehe'], row['Bbox_Breite'])
        print('Ratio:', ratio)
        # TODO: Use MassFunction? to evaluate the result
        
        
        #def calcSpeed(oldOwnSpeed, newOwnSpeed, oldDistance, newDistance):
        if(index==0):
            continue
            
        speed = calcSpeed(data.loc[(index - 1),'Eigengeschwindigkeit(m/s)'],
                            row['Eigengeschwindigkeit(m/s)'],
                            data.loc[(index - 1),'Abstand(m)'],
                            row['Abstand(m)'])
        print('Geschwindigkeit: ', speed)
        
    
iterateMeasurement(data)

---------------------------------------------------
Zeitpunkt: 0.0
Ratio: 1.1
---------------------------------------------------
Zeitpunkt: 0.1
Ratio: 1.1111111111111112
Geschwindigkeit:  27.1
---------------------------------------------------
Zeitpunkt: 0.2
Ratio: 1.1046511627906976
Geschwindigkeit:  22.2
---------------------------------------------------
Zeitpunkt: 0.3
Ratio: 1.1111111111111112
Geschwindigkeit:  17.15
---------------------------------------------------
Zeitpunkt: 0.4
Ratio: 1.1111111111111112
Geschwindigkeit:  22.1
---------------------------------------------------
Zeitpunkt: 0.5
Ratio: 1.10752688172043
Geschwindigkeit:  22.05
---------------------------------------------------
Zeitpunkt: 0.6
Ratio: 1.10752688172043
Geschwindigkeit:  17.05
---------------------------------------------------
Zeitpunkt: 0.7
Ratio: 1.1041666666666667
Geschwindigkeit:  7.149999999999999
---------------------------------------------------
Zeitpunkt: 0.8
Ratio: 1.1063829787234043
Gesch

Geschwindigkeit:  22.15
---------------------------------------------------
Zeitpunkt: 8.1
Ratio: 1.1048951048951048
Geschwindigkeit:  17.15
---------------------------------------------------
Zeitpunkt: 8.2
Ratio: 0.8913043478260869
Geschwindigkeit:  27.2
---------------------------------------------------
Zeitpunkt: 8.3
Ratio: 1.1095890410958904
Geschwindigkeit:  22.2
---------------------------------------------------
Zeitpunkt: 8.4
Ratio: 1.1095890410958904
Geschwindigkeit:  12.2
---------------------------------------------------
Zeitpunkt: 8.5
Ratio: 1.1066666666666667
Geschwindigkeit:  17.15
---------------------------------------------------
Zeitpunkt: 8.6
Ratio: 1.110344827586207
Geschwindigkeit:  27.05
---------------------------------------------------
Zeitpunkt: 8.7
Ratio: 1.1095890410958904
Geschwindigkeit:  22.05
---------------------------------------------------
Zeitpunkt: 8.8
Ratio: 1.105263157894737
Geschwindigkeit:  12.100000000000001
--------------------------------